https://www.youtube.com/watch?v=T0BiFBaMLDQ&t=215s

## Preprocessing

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob
import imageio
# from albumentations import HorizontalFlip, VerticalFlip, Rotate # !!!!
from tqdm import tqdm
from torch.utils.data import DataLoader,Dataset

In [2]:
"""儲存csv"""
# import pandas as pd
# a=pd.DataFrame(X1)
# a.to_csv("X1_ground.csv")
"""Load npy檔"""
# Y1 = np.load(os.path.join(dataset,"Y1_predict.npy"))
"""Save npy檔"""
# np.save("data_train.npy",data_train)
"""模型"""
def save_model(model,filename):
    state = model.state_dict()
    for key in state: state[key] = state[key].clone().cpu()
    torch.save(state, filename)
    
def load_model(model,filename):
    model.load_state_dict(torch.load(filename))
    return model

In [3]:
# !unzip dataset.zip

In [4]:
# Create a directory
def create_directory(path):
  if not os.path.exists(path):
    os.makedirs(path)

def load_data(path):
  train_x = sorted(glob(os.path.join(path,'train','images','*.tif'))) # training photo
  train_y = sorted(glob(os.path.join(path,'train','mask','*.gif'))) # training mask

  test_x = sorted(glob(os.path.join(path,'test','images','*.tif'))) # training photo
  test_y = sorted(glob(os.path.join(path,'test','mask','*.gif'))) # training mask

  return ((train_x, train_y), (test_x,test_y))

In [5]:
def augment_data(images, masks, save_path, augment=True):
  size = (512, 512)
  
  for idx, (x,y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
    # 抽出必要的字詞
    name = x.split('/')[-1].split('.')[0] # ['', 'content', 'dataset', 'train', 'images', '21_training.tif'] 負一 只取最後一項
    
    # read image and mask
    x = cv2.imread(x,cv2.IMREAD_COLOR) # (584, 565, 3)
    y = imageio.mimread(y)[0] # (584, 565)

    # 處理augmentation
    if augment == True:
      aug = HorizontalFlip(p=1.0)
      augmented = aug(image=x, mask=y)
      x1 = augmented['image']
      y1 = augmented['mask']

      aug = VerticalFlip(p=1.0)
      augmented = aug(image=x, mask=y)
      x2 = augmented['image']
      y2 = augmented['mask']

      aug = Rotate(limit=45,p=1.0) # degree for rotation !!!
      augmented = aug(image=x, mask=y)
      x3 = augmented['image']
      y3 = augmented['mask']

      X = [x,x1,x2,x3]
      Y = [y,y1,y2,y3]

      pass

    else:
      X = [x]  ### ????
      Y = [y]

    # 重新命名放入new_data資料夾
    index = 0
    for i,j in zip(X,Y):
      i = cv2.resize(i,size) # 改變大小
      j = cv2.resize(j,size)

      tmp_img = f'{name}_{index}.png' # 21_training_0.png
      tmp_mask = f'{name}_{index}.png'

      image_path = os.path.join(save_path,'images',tmp_img) 
      mask_path = os.path.join(save_path,'mask',tmp_mask)

      cv2.imwrite(image_path, i) # 寫入資料夾
      cv2.imwrite(mask_path, j)

      index +=1

In [6]:
if __name__=='__main__':
  # Random Seed
  np.random.seed(42)
  # Load Data
  data_path = '/content/dataset'
  ((train_x, train_y), (test_x,test_y)) = load_data(data_path)

  print('datasest files len:',len(train_x),len(train_y),len(test_x),len(test_y))

  # Create folder for new augmentation data
  create_directory('/content/new_data/train/images/')
  create_directory('/content/new_data/train/mask/')
  create_directory('/content/new_data/test/images/')
  create_directory('/content/new_data/test/mask/')

  # Do augmentation
  augment_data(train_x, train_y, '/content/new_data/train/', augment= True)
  augment_data(test_x, test_y, '/content/new_data/test/', augment= False) # 為甚麼不用test_x,y???


In [7]:
# !zip -r /content/new_data.zip /content/new_data

## UNet

In [8]:
import torch
import torch.nn as nn

In [9]:
class conv_block(nn.Module):
  def __init__(self, in_c, out_c):
    super().__init__()

    self.conv1 = nn.Conv2d(in_c, out_c, kernel_size=3, padding=1)
    self.bn1 = nn.BatchNorm2d(out_c)

    self.conv2 = nn.Conv2d(out_c, out_c, kernel_size=3, padding=1)
    self.bn2 = nn.BatchNorm2d(out_c)
    
    self.relu = nn.ReLU()

  def forward(self, inputs):
    x = self.conv1(inputs) # [2, 64, 128, 128] 奇怪kernel size 不是3嗎
    x = self.bn1(x)
    x = self.relu(x)

    x = self.conv2(x) # [2, 64, 128, 128]
    x = self.bn2(x)
    x = self.relu(x)

    return x

class encoder_block(nn.Module):
  def __init__(self, in_c, out_c):
    super().__init__()

    self.conv = conv_block(in_c, out_c)
    self.pool = nn.MaxPool2d((2,2)) 

  def forward(self, inputs):
    x = self.conv(inputs)
    p = self.pool(x) 

    return x, p

class decoder_block(nn.Module):
  def __init__(self, in_c, out_c):
    super().__init__()

    self.up = nn.ConvTranspose2d(in_c, out_c, kernel_size=2, stride=2, padding=0) # upsampling?
    self.conv = conv_block(out_c+out_c, out_c) # 加上encoder的layer

  def forward(self, inputs, skip):
    x = self.up(inputs)
    x = torch.cat([x, skip], axis=1) # 加上encoder的layer # pytorch第一軸是channel (tensorflow第-1軸)
    x = self.conv(x)
    return x

class build_unet(nn.Module):
  def __init__(self):
    super().__init__()

    # Encoder
    self.e1 = encoder_block(3,64)
    self.e2 = encoder_block(64,128)
    self.e3 = encoder_block(128,256)
    self.e4 = encoder_block(256,512)

    # BottleNeck
    self.b = conv_block(512, 1024)

    # Decoder
    self.d1 = decoder_block(1024, 512)
    self.d2 = decoder_block(512, 256)
    self.d3 = decoder_block(256, 128)
    self.d4 = decoder_block(128, 64)

    # Classifier 二元分類 channel=1
    self.outputs = nn.Conv2d(64, 1, kernel_size=1, padding=0)


  def forward(self, inputs):
    # Encoder
    s1, p1 = self.e1(inputs) # [2, 64, 512, 512]
    s2, p2 = self.e2(p1) # [2, 128, 256, 256]
    s3, p3 = self.e3(p2) # [2, 256, 128, 128]
    s4, p4 = self.e4(p3) # [2, 512, 64, 64]

    # BottleNeck
    b = self.b(p4) # [2, 1024, 32, 32]

    """ Decoder """
    d1 = self.d1(b, s4) # s for skip block
    d2 = self.d2(d1, s3)
    d3 = self.d3(d2, s2)
    d4 = self.d4(d3, s1)

    outputs = self.outputs(d4)
    # print(outputs.shape)


    # print(s1.shape,s2.shape,s3.shape,s4.shape,b.shape)


    return outputs
    

In [ ]:
def conv_layer(chann_in, chann_out, k_size, p_size):
    layer = nn.Sequential(
        nn.Conv2d(chann_in, chann_out, kernel_size=k_size, padding=p_size),
        nn.BatchNorm2d(chann_out),
        nn.ReLU()
        # nn.Sigmoid()
    )
    return layer

def vgg_conv_block(in_list, out_list, k_list, p_list, pooling_k, pooling_s):

    layers = [ conv_layer(in_list[i], out_list[i], k_list[i], p_list[i]) for i in range(len(in_list)) ]
    # 2, 64, 512, 512
    # layers = layers + [ nn.MaxPool2d(kernel_size = pooling_k, stride = pooling_s)]
    # 2, 64, 256, 256

    return nn.Sequential(*layers) #  list-like layer

def vgg_fc_layer(size_in, size_out):
    layer = nn.Sequential(
        nn.Linear(size_in, size_out),
        nn.BatchNorm1d(size_out),
        nn.ReLU()
        # nn.Sigmoid()
    )
    return layer

class VGG16(nn.Module):
    def __init__(self, n_classes=2):
        super(VGG16, self).__init__()

        # Conv blocks (BatchNorm + ReLU activation added in each block)
        self.layer1 = vgg_conv_block([3,64], [64,64], [3,3], [1,1], 2, 2)
        self.max1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.layer2 = vgg_conv_block([64,128], [128,128], [3,3], [1,1], 2, 2)
        self.max2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.layer3 = vgg_conv_block([128,256,256], [256,256,256], [3,3,3], [1,1,1], 2, 2)
        self.max3 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.layer4 = vgg_conv_block([256,512,512], [512,512,512], [3,3,3], [1,1,1], 2, 2)
        self.max4 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.layer5 = vgg_conv_block([512,512,512], [512,512,512], [3,3,3], [1,1,1], 2, 2)
        self.max5 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        # FC layers
        self.layer6 = vgg_fc_layer(16*16*512, 4096) # 長x寬xfeature 數
        self.layer7 = vgg_fc_layer(4096, 4096)

        # Interpolation
        self.inter = Interpolate(size=(512, 512), mode='bilinear')


        # Final layer
        self.layer8 = nn.Linear(4096, n_classes) # 後面我自己加的
        self.outputs = nn.Conv2d(384, 1, kernel_size=1, padding=0)



    # original VGG16
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        vgg16_features = self.layer5(out)
        print(vgg16_features.shape)
        out = vgg16_features.view(out.size(0), -1)
        out = self.layer6(out) # ([2, 4096])
        out = self.layer7(out)
        out = self.layer8(out)
        # out = self.layer9(out)
        # out = self.layer10(out)
        # (2, 2) --> ([2, 1, 512, 512])

        return  out # ,vgg16_features

In [11]:
#  if __name__== '__main__':
#    x = torch.randn((2,3,512,512)) # batch_size, channel, image size, sizes
#    f = VGG16()
#    f(x)

## Loss

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score

In [14]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = (0.6*BCE + 0.4*dice_loss) # 因為BCE小，DICE大，調整至兩者相同

        return Dice_BCE

class BCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(BCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')

        return BCE

##utils

In [15]:
import os
import time
import random
import numpy as np
import cv2
import torch

""" Seeding the randomnseed. """
def seeding(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

""" Create a directory. """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

""" Calculate the time taken """
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Data.py

In [16]:
class DriveDataset(Dataset):
    def __init__(self, images_path, masks_path):
        self.images_path = images_path
        self.masks_path = masks_path
        self.n_smaples = len(images_path)

    def __getitem__(self, index):
        # Read Image
        image = cv2.imread(self.images_path[index],cv2.IMREAD_COLOR)
        image = image/255.0 # [512, 512, 3]
        image = np.transpose(image, (2,0,1)) # [3, 512, 512]
        image = image.astype(np.float32)
        image = torch.from_numpy(image)

        # Read Mask
        mask = cv2.imread(self.masks_path[index],cv2.IMREAD_GRAYSCALE)
        mask = mask/255.0 # [512, 512]
        mask = np.expand_dims(mask, axis=0) # [1, 512, 512]
        mask = mask.astype(np.float32)
        mask = torch.from_numpy(mask)

        return image, mask

    def __len__(self):
        return self.n_smaples

## Training

In [17]:
import os
import time
from glob import glob
import cv2

import torch
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn

In [18]:
# !unzip new_data.zip

In [20]:
# m = nn.Sigmoid()
# loss1 = nn.BCELoss()
# input1 = torch.randn((3,2), requires_grad=True)
# target1 = torch.empty((3,2)).random_(2)
# output1 = loss1(m(input1), target1)
# output1.backward()
# print(output1.item())

In [21]:
def train(model, loader, optimizer, loss_fn, device):
  epoch_loss = 0.0

  model.train() # not test, not evaluate
  for x, y in loader:
    x = x.to(device, dtype = torch.float32)
    y = y.to(device, dtype = torch.float32)

    # Sets the gradients of all optimized torch.Tensor s to zero.
    # 每次更新weights,bias後歸零
    optimizer.zero_grad() 
    y_prediction = model(x) # 模型內沒有sigmoid這邊要嗎?
    loss = loss_fn(y_prediction, y)
    loss.backward()
    optimizer.step()
    epoch_loss +=loss.item()
    # print(y.shape) # 2,1,512,512
    # print(y_prediction.shape) # 2,1,512,512

  epoch_loss = epoch_loss/len(loader)

  return epoch_loss

def evaluate(model, loader, loss_fn, device):
  epoch_loss = 0.0

  model.eval()
  with torch.no_grad():
    for x, y in loader:
     x = x.to(device, dtype = torch.float32)
     y = y.to(device, dtype = torch.float32)
     
     y_prediction = model(x)
     loss = loss_fn(y_prediction, y)
     epoch_loss +=loss.item()
    
    epoch_loss = epoch_loss/len(loader)

  return epoch_loss

Start Training

In [23]:
if __name__=="__main__":
    # fixed
    seeding(42)

    # file
    create_dir('C:/Users/User/Desktop/UNet/files')


    # create_dir('/content/files')

    # Load data
    train_x = sorted(glob("C://Users//User//Desktop//UNet//new_data//train//images//*"))
    train_y = sorted(glob("C://Users//User//Desktop//UNet//new_data//train//mask//*"))
    valid_x = sorted(glob("C://Users//User//Desktop//UNet//new_data//test//images//*"))
    valid_y = sorted(glob("C://Users//User//Desktop//UNet//new_data//test//mask//*"))

    # train_x = sorted(glob("../content/new_data/train/images/*"))
    # train_y = sorted(glob("../content/new_data/train/mask/*"))
    # valid_x = sorted(glob("../content/new_data/test/images/*"))
    # valid_y = sorted(glob("../content/new_data/test/mask/*"))

    # Check data size
    data_str = f"Size:\nTrain: {len(train_x)} - Valid: {len(valid_x)}"
    print(data_str)

    # Hyperparameters
    H,W = 512,512
    size = (H,W)
    batch_size = 2
    num_epoch = 50
    lr =  1e-4 # UNet 1e-4 # VGG 1e-3
    checkpoint_path = "C:/Users/User/Desktop/UNet/files/checkpoint.pth"

    # Dataset and Loader
    train_dataset = DriveDataset(train_x, train_y)
    valid_dataset = DriveDataset(valid_x, valid_y)

    train_loader = DataLoader(
      dataset = train_dataset,
      batch_size = batch_size,
      shuffle = True,
      num_workers = 0
    )


    valid_loader = DataLoader(
      dataset = valid_dataset,
      batch_size = batch_size,
      shuffle = False,
      num_workers = 0
    )

    # device = torch.device('cpu') # 'cuda'
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    model = build_unet()
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)

    # loss_fn = BCELoss()
    loss_fn = DiceLoss()
    # loss_fn = nn.BCEWithLogitsLoss()
    # loss_fn = DiceBCELoss()


    loss_list_train =[]
    loss_list_val =[]


    # Strat Training
    best_valid_loss = float('inf')
    for epoch in range(num_epoch):
        start_time = time.time()

        train_loss = train(model, train_loader, optimizer, loss_fn, device)
        valid_loss = evaluate(model, valid_loader, loss_fn, device)

        loss_list_train.append(train_loss)
        loss_list_val.append(valid_loss)


        # Save better model
        if valid_loss < best_valid_loss:
            data_str = f'Valid Loss improved from {best_valid_loss:2.4f} to {valid_loss:2.4f}'
            print(data_str)

            best_valid_loss = valid_loss
            torch.save(model.state_dict(), checkpoint_path)

        end_time = time.time()
        mins, secs = epoch_time(start_time, end_time)

        data_str = f'Epoch: {epoch+1:02} | Epoch Time: {mins}m {secs}s\n'
        data_str += f'\tTrain Loss: {train_loss:.5f}'
        data_str += f'\tVal Loss: {valid_loss:.5f}\n'
        print(data_str)
      

Size:
Train: 80 - Valid: 20
cuda
Valid Loss improved from inf to 0.8422
Epoch: 01 | Epoch Time: 0m 10s
	Train Loss: 0.76374	Val Loss: 0.84222

Valid Loss improved from 0.8422 to 0.6172
Epoch: 02 | Epoch Time: 0m 8s
	Train Loss: 0.69183	Val Loss: 0.61725

Valid Loss improved from 0.6172 to 0.5411
Epoch: 03 | Epoch Time: 0m 8s
	Train Loss: 0.60514	Val Loss: 0.54109

Valid Loss improved from 0.5411 to 0.4270
Epoch: 04 | Epoch Time: 0m 8s
	Train Loss: 0.54747	Val Loss: 0.42701

Valid Loss improved from 0.4270 to 0.4034
Epoch: 05 | Epoch Time: 0m 8s
	Train Loss: 0.50755	Val Loss: 0.40345

Valid Loss improved from 0.4034 to 0.3603
Epoch: 06 | Epoch Time: 0m 8s
	Train Loss: 0.47733	Val Loss: 0.36027

Valid Loss improved from 0.3603 to 0.3307
Epoch: 07 | Epoch Time: 0m 8s
	Train Loss: 0.46112	Val Loss: 0.33069

Valid Loss improved from 0.3307 to 0.3115
Epoch: 08 | Epoch Time: 0m 8s
	Train Loss: 0.44569	Val Loss: 0.31152

Valid Loss improved from 0.3115 to 0.3019
Epoch: 09 | Epoch Time: 0m 8s
	

In [ ]:
import csv

with open('VGG_mixed.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(zip(loss_list_train, loss_list_val))

## Test.py

In [ ]:
import os, time
from operator import add
import numpy as np
from glob import glob
import cv2
from tqdm import tqdm
import imageio
import torch
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score

In [ ]:
def calculate_metrics(y_true, y_pred):
    """ Ground truth """
    y_true = y_true.cpu().numpy()
    y_true = y_true > 0.5  # 把>0.5的當作有，只計算有的(偽陽性)，或許可以計算沒有的
    y_true = y_true.astype(np.uint8)
    y_true = y_true.reshape(-1)

    """ Prediction """
    y_pred = y_pred.cpu().numpy()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.uint8)
    y_pred = y_pred.reshape(-1)

    score_jaccard = jaccard_score(y_true, y_pred)
    score_f1 = f1_score(y_true, y_pred)
    score_recall = recall_score(y_true, y_pred)
    score_precision = precision_score(y_true, y_pred)
    score_acc = accuracy_score(y_true, y_pred)

    return [score_jaccard, score_f1, score_recall, score_precision, score_acc]

def mask_parse(mask):
    mask = np.expand_dims(mask, axis=-1)    ## (512, 512, 1)
    mask = np.concatenate([mask, mask, mask], axis=-1)  ## (512, 512, 3) # 和彩色圖片(test data)同樣維度
    return mask

In [ ]:
if __name__=="__main__":
    # fixed
    seeding(42)

    # file
    create_dir('C:/Users/User/Desktop/UNet/results')

    # create_dir('/content/results')

    # Testing set
    test_x = sorted(glob("C:/Users/User/Desktop/UNet/new_data/test/images/*"))
    test_y = sorted(glob("C:/Users/User/Desktop/UNet/new_data/test/mask/*"))


    # test_x = sorted(glob("../content/new_data/test/images/*"))
    # test_y = sorted(glob("../content/new_data/test/mask/*"))

    # Hyperparameters
    H,W = 512,512
    size = (H,W)
    checkpoint_path = "C:/Users/User/Desktop/UNet/files/checkpoint.pth"

    # Load checkpoint
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = build_unet()
    model = model.to(device)
    model.load_state_dict((torch.load(checkpoint_path, map_location = device))) # 載入訓練好的模型
    model.eval() # Set the mode

    metrics_score = [0,0,0,0,0]
    time_taken = []

    for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
        """ Extract the name """
        name = x.split("\\")[-1].split(".")[0]

        """ Read image"""
        image = cv2.imread(x, cv2.IMREAD_COLOR) ## (512, 512, 3)
        x = np.transpose(image, (2, 0, 1)) ## (3, 512, 512)
        x = x/255.0
        x = np.expand_dims(x, axis=0) ## (1, 3, 512, 512) 為了在一個batch裡面?
        x = x.astype(np.float32)
        x = torch.from_numpy(x)
        x = x.to(device)

        """ Reading mask """
        mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
        y = np.expand_dims(mask, axis=0) ## (1, 512, 512)
        y = y/255.0
        y = np.expand_dims(y, axis=0) ## (1, 1, 512, 512)
        y = y.astype(np.float32)
        y = torch.from_numpy(y)
        y = y.to(device)


        with torch.no_grad():
            """ Prediction and Calculating FPS """
            start_time = time.time()
            pred_y = model(x)
            pred_y = torch.sigmoid(pred_y) # 其他都沒加這個
            total_time = time.time() - start_time
            time_taken.append(total_time)

            score = calculate_metrics(y, pred_y)
            metrics_score = list(map(add, metrics_score, score))
            
            pred_y = pred_y[0].cpu().numpy() ## (1, 512, 512)
            pred_y = np.squeeze(pred_y, axis=0) ## (512, 512)
            pred_y = pred_y > 0.5
            pred_y = np.array(pred_y, dtype=np.uint8)

            """ Saving masks """
            ori_mask = mask_parse(mask) # 正確
            pred_y = mask_parse(pred_y) # 預測
            line = np.ones((size[1], 10, 3)) * 128 # 產生一條線

            cat_images = np.concatenate(
                [image, line, ori_mask, line, pred_y * 255], axis=1
            )
            # 儲存圖片
            cv2.imwrite(f"C:/Users/User/Desktop/UNet/results/{name}.png", cat_images)


    # 計算平均分數
    jaccard = metrics_score[0]/len(test_x)
    f1 = metrics_score[1]/len(test_x)
    recall = metrics_score[2]/len(test_x)
    precision = metrics_score[3]/len(test_x)
    acc = metrics_score[4]/len(test_x)
    print(f"Jaccard: {jaccard:1.4f} - F1: {f1:1.4f} - Recall: {recall:1.4f} - Precision: {precision:1.4f} - Acc: {acc:1.4f}")

    fps = 1/np.mean(time_taken)
    print("FPS: ", fps)

    scores = [jaccard, f1, recall, precision, acc]
    np.savetxt("C:/Users/User/Desktop/UNet/results/metrics_score.csv", scores, delimiter =",",fmt ='% s')

100%|██████████| 20/20 [00:04<00:00,  4.40it/s]

Jaccard: 0.6560 - F1: 0.7917 - Recall: 0.7773 - Precision: 0.8154 - Acc: 0.9646
FPS:  180.68538294856828
